In [3]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as p

sagarin_page = 'http://sagarin.com/sports/cbsend.htm#New_Feature'

request = urllib.request.Request(sagarin_page)
response = urllib.request.urlopen(request)
page = response.read().decode('utf-8')

soup = BeautifulSoup(page, 'html.parser')
content = soup.find_all('pre')[1].text.strip()

#http://usatoday30.usatoday.com/sports/sagarin/bkt1213.htm
lineparts = ["Rank","Team","Rating","W","L","Schedule","ScheduleRank","WinsVsTop25","LossesVsTop25","WinsVsTop50","LossesVsTop50","ELO_SCORE","ELO_RANK","PREDICTOR","PREDICTOR_RANK"]

def sanitize_teamname(name):
    result = name.replace('State','St.').replace('NC ','North Carolina ').replace('Saint','St.')
    result = result.replace('Albany-NY','Albany')
    result = result.replace('VCU(Va. Commonwealth)','Virginia Commonwealth')
    result = result.replace('Southern U.','Southern')
    result = result.replace('UNLV','Nevada Las Vegas')
    result = result.replace('Miami-Florida','Miami FL')
    result = result.replace("St. Mary's-Cal.","St. Mary's")
    return result
#print sanitize_teamname("Saint MAry's-Cal.")

sagarindata = {}
data = {
    'Team':[],
    'Rank':[],
    'Rating':[]
}

for line in content.split('\n'):

    if not line.strip():
        continue
    parts = line.strip().split()
    if not parts:
        continue
    if parts[0] in ["RATING","HOME","College", "____________________________________________________________________________________________________________________________________"]:
        continue
    left,right = line.split('=')
    rank = left.split()[0]
    team = ' '.join(left.split()[1:])
    rank = int(rank)
    team = sanitize_teamname(team)
    if team not in sagarindata:
        sagarindata[team] = {}
    sagarindata[team]['Team'] = team
    sagarindata[team]['Rank'] = rank
    data['Team'].append(team)
    data['Rank'].append(rank)
    data['Rating'].append(float(right.split()[0]))


df = p.DataFrame(data)

df.set_index('Rank')
df.to_csv('sagarin.csv')